In [1]:
import os
import pandas as pd
import warnings

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier

## Loading Data

In [2]:
#Set training and test folder paths
training_path = 'features_train/features_train'
test_path = 'features_test/features_test'

#Load labels file
labels = pd.read_csv('labels.csv')

#Load feature description files, take out column 0 to use as header for training/test sets
features = pd.read_csv('feature_description.csv', encoding_errors='ignore', header=None, index_col=0)
features = features.index.tolist()

In [3]:
def load_data(folder_path):
    #Init empty dataframe
    res = pd.DataFrame()
    for file in os.listdir(folder_path):
        #for each speaker file
        if file.endswith('.csv'):
            #get participant id from filename, eg filename: 'spk_305.csv'
            participant = float(file.split('_')[1].split('.')[0])
            #find labels for the participant
            label = labels[labels['Participant_ID'] == participant]
            #load participant feature file
            file_path = os.path.join(folder_path, file)
            data_df = pd.read_csv(file_path, header=None, names=features)
            #Add labels and participant id columns
            data_df['participant'] = participant
            data_df['gender'] = label['Gender'].values[0]
            data_df['depression'] = label['Depression'].values[0]
            #combine everything to result
            res = pd.concat([res, data_df])
    return res

In [4]:
#Load training data
training_df = load_data(training_path)
len(training_df)

13626

In [5]:
#Load test data
test_df = load_data(test_path)
len(test_df)

3280

## Data cleaning and preprocessing 

In [6]:
# Check Missing values
missing_values = (training_df.isnull().sum()/len(training_df)) *100
total_missing_values = training_df.isnull().any(axis=1).sum()
print(f'Missing value percent % for each column, total samples {len(training_df)}')
print(f'Number of samples with missing values: {total_missing_values}')
print(missing_values)

Missing value percent % for each column, total samples 13626
Number of samples with missing values: 1
F0semitoneFrom27.5Hz_sma3nz_amean             0.007339
F0semitoneFrom27.5Hz_sma3nz_stddevNorm        0.007339
F0semitoneFrom27.5Hz_sma3nz_percentile20.0    0.007339
F0semitoneFrom27.5Hz_sma3nz_percentile50.0    0.007339
F0semitoneFrom27.5Hz_sma3nz_percentile80.0    0.007339
                                                ...   
StddevUnvoicedSegmentLength                   0.007339
equivalentSoundLevel_dBp                      0.007339
participant                                   0.000000
gender                                        0.000000
depression                                    0.000000
Length: 91, dtype: float64


In [7]:
def preprocess_data(data):
    #drop missing va;ues since there is just 1 sample
    data_nona = data.dropna()
    #Normalization z-score
    scaler = StandardScaler()    
    # Store the columns to keep for later concatenation
    columns_to_keep = ['participant', 'gender', 'depression']
    # Extract the columns to be scaled and drop them from the original DataFrame
    temp = data_nona[columns_to_keep].copy()
    data_nona.drop(columns=columns_to_keep, axis=1, inplace=True)
    # Scale the remaining columns using StandardScaler and convert back to DataFrame
    scaled_data = pd.DataFrame(scaler.fit_transform(data_nona), columns=data_nona.columns)
    scaled_data.reset_index(drop=True, inplace=True)
    temp.reset_index(drop=True, inplace=True)
    # Concatenate the scaled data with the columns we kept earlier
    processed_data = pd.concat([scaled_data, temp], axis=1)
    return processed_data

## Methods to calculate metrics

In [8]:
# Calculates accuracy
# pass true and predicted labels
# return accuracy score
def calculate_total_accuracy(true_labels, predicted_labels):
    return accuracy_score(true_labels, predicted_labels)

# Calculates accuracy
# pass true and predicted labels
# return balanced accuracy score
def calculate_balanced_accuracy(true_labels, predicted_labels):
    #calculkate confusion matrix
    matrix = confusion_matrix(true_labels, predicted_labels)
    TP = matrix[1, 1]
    TN = matrix[0, 0]
    FP = matrix[0, 1]
    FN = matrix[1, 0]
    #For positive class, how many correct predictions
    accuracy_positive = TP/(TP+FN)
    #For negative class how many 
    accuracy_negative = TN/(TN+FP)
    return 0.5*(accuracy_positive + accuracy_negative)

#Calculates Equality of Opportunity
# pass true and predicted labels for male samples
# pass true and predicted labels for female samples
# return balanced accuracy score
def calculate_EO(true_labels_male, 
                 true_labels_female,
                 predicted_labels_male,
                predicted_labels_female):
    #Calculate True pistive rate for male gender with confusion matrix
    matrix_male = confusion_matrix(true_labels_male, predicted_labels_male)
    TP = matrix_male[1, 1]
    TN = matrix_male[0, 0]
    FN = matrix_male[1, 0]
    TPR_male = TP/(TP+FN)

    #Calculate True pistive rate for female gender with confusion matrix
    matrix_female = confusion_matrix(true_labels_female, predicted_labels_female)
    TP = matrix_female[1, 1]
    TN = matrix_female[0, 0]
    FN = matrix_female[1, 0]
    TPR_female = TP/(TP+FN)
    
    # Calculate EO
    return 1-abs(TPR_male-TPR_female) 

#Function to calculate majority votings
#Pass labels
#Returns mode or which label was predicted most
def majority_voting(df):
    counts = df.value_counts()
    return counts.idxmax()

In [9]:
#Function to calculate all metrics
#Pass true labels, predicted labels and a reference(test/val) dataframe
#referece dataframe should have all labels and features
#Returns a dictionary with all the metric calculated
def calculate_metrics(y_true, y_pred, test_data, EO=True):
    # Initialize metrics
    metrics = {}
    #---------------------------------------------------CALCULATING TOTAL METRICS
    #calculate total accuracy
    metrics["Total accuracy"] = calculate_total_accuracy(y_true, y_pred)
    #calculate total balanced accuracy
    metrics["Total Balanced accuracy"] = calculate_balanced_accuracy(y_true, y_pred)
    #calculate total EO
    if(EO):
        #find gender based indices for true labels from data
        male_indices = test_data[test_data['gender']==1].index
        female_indices = test_data[test_data['gender']==0].index
        #separate true labels based on indices
        male_true = y_true.loc[male_indices]
        female_true = y_true.loc[female_indices]
        #Find the corresponding indices for predicted labels from true_labels
        male_true_index_list = male_true.index.tolist()
        female_true_index_list = female_true.index.tolist()
        #Get separated predicted labels based on gender
        male_predicted = y_pred[[male_true_index_list.index(index) for index in male_true_index_list]]
        female_predicted = y_pred[[female_true_index_list.index(index) for index in female_true_index_list]]
        metrics["Total EO"] = calculate_EO(male_true, female_true, male_predicted, female_predicted)
    #-------------------------------------------------CALCULATING AGGREGATED METRICS FOR EACH PARTICIPANT
    predictions_df = pd.DataFrame({'participant': test_data['participant'], 'predicted_label': y_pred, 'true_label': y_true})
    aggregated_y_true = predictions_df.groupby('participant')['true_label'].agg(majority_voting)
    aggregated_y_pred = predictions_df.groupby('participant')['predicted_label'].agg(majority_voting)
    #Calculate aggregated accuracy score
    metrics["Aggregated accuracy score"] = calculate_total_accuracy(aggregated_y_true, aggregated_y_pred)
    #Calculate balanced aggregated accuracy
    metrics["Aggregated balanced accuracy score"] = calculate_balanced_accuracy(aggregated_y_true, aggregated_y_pred)
    if(EO):
        #Calculate aggregated EOs
        male_predictions_df = pd.DataFrame({'participant': test_data['participant'].loc[male_indices], 'predicted_label': y_pred, 'true_label': y_true})
        male_aggregated_y_true = male_predictions_df.groupby('participant')['true_label'].agg(majority_voting)
        male_aggregated_y_pred = male_predictions_df.groupby('participant')['predicted_label'].agg(majority_voting)
        female_predictions_df = pd.DataFrame({'participant': test_data['participant'].loc[female_indices], 'predicted_label': y_pred, 'true_label': y_true})
        female_aggregated_y_true = female_predictions_df.groupby('participant')['true_label'].agg(majority_voting)
        female_aggregated_y_pred = female_predictions_df.groupby('participant')['predicted_label'].agg(majority_voting)
        metrics["Aggregated EO score"] = calculate_EO(male_aggregated_y_true, female_aggregated_y_true, male_aggregated_y_pred, female_aggregated_y_pred)
        metrics["Aggregated balanced accuracy score (male)"] = calculate_balanced_accuracy(male_aggregated_y_true, male_aggregated_y_pred)
        metrics["Aggregated balanced accuracy score (female)"] = calculate_balanced_accuracy(female_aggregated_y_true, female_aggregated_y_pred)
    return metrics

## Data Modeling - Depression Classification
### What models to try?
- Decision tree
- Random forest
- Logistic Regression
- Adaboost Classifier
- Linear Perceptron

### Model attempt: Decision tree classifier

In [10]:
#------------------------------------------------Tuning for different depths-------------------------------------------------------

warnings.filterwarnings("ignore")
training_data = preprocess_data(training_df)
X = training_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y = training_data['depression']

# Define the depths to experiment with
depths = [3, 5, 7, 9, 15, 30, 50, 70, 90]
# Initialize metrics
metrics = {}

# Perform cross-validation for each tree depth
for depth in depths:
    # Initialize decision tree model
    tree = DecisionTreeClassifier(max_depth=depth, criterion='entropy')
    # Cross validation k fold, 4:1::training:validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    #list to store metrics for each cross validation split
    fold_metrics = []

    # Perform cross-validation and collect metrics
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        #Fit the training set
        tree.fit(X_train, y_train)
        #Predict validation set
        y_pred = tree.predict(X_val)
        #calculate metrics
        fold_metrics.append(calculate_metrics(y_val, y_pred, training_data.iloc[val_index]))

    #find avg metrics for each depth
    metrics[depth] = {}
    sums = {}
    for metric in fold_metrics:
        for key, value in metric.items():
            sums[key] = sums.get(key, 0) + value
    for key in sums:
        sums[key]/=len(fold_metrics)
    metrics[depth] = sums

#Print metrics for all the hyperparameters (Depth)
for depth in metrics:
    print(f"for depth {depth}")
    print(metrics[depth])
    print()

for depth 3
{'Total accuracy': 0.739302752293578, 'Total Balanced accuracy': 0.5909240479132576, 'Total EO': 0.9504593123805108, 'Aggregated accuracy score': 0.7586206896551724, 'Aggregated balanced accuracy score': 0.5857142857142856, 'Aggregated EO score': 0.8333333333333334, 'Aggregated balanced accuracy score (male)': 0.5750000000000001, 'Aggregated balanced accuracy score (female)': 0.5875}

for depth 5
{'Total accuracy': 0.7470825688073395, 'Total Balanced accuracy': 0.613583936397171, 'Total EO': 0.9324445056229891, 'Aggregated accuracy score': 0.7839080459770115, 'Aggregated balanced accuracy score': 0.6109126984126985, 'Aggregated EO score': 0.8833333333333332, 'Aggregated balanced accuracy score (male)': 0.6083333333333334, 'Aggregated balanced accuracy score (female)': 0.6125}

for depth 7
{'Total accuracy': 0.7634495412844037, 'Total Balanced accuracy': 0.6655559677154813, 'Total EO': 0.9631518642013741, 'Aggregated accuracy score': 0.8137931034482758, 'Aggregated balanced 

In [11]:
# ---------------------------------------------------------Testing for best depth----------------------------------------------------------------
test_data = preprocess_data(test_df)
training_data = preprocess_data(training_df)
X_train = training_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y_train = training_data['depression']
X_test = test_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y_test = test_data['depression']

best_depth = 70
# Initialize decision tree model
tree = DecisionTreeClassifier(max_depth=best_depth, criterion='entropy')
#Fit the training set
tree.fit(X_train, y_train)
#Predict for test set
y_pred = tree.predict(X_test)
#get metrics
metrics = calculate_metrics(y_test, y_pred, test_data)

print(metrics)

{'Total accuracy': 0.688109756097561, 'Total Balanced accuracy': 0.5868557627337944, 'Total EO': 0.7612878309107507, 'Aggregated accuracy score': 0.7, 'Aggregated balanced accuracy score': 0.5476190476190477, 'Aggregated EO score': 0.8, 'Aggregated balanced accuracy score (male)': 0.45454545454545453, 'Aggregated balanced accuracy score (female)': 0.6}


## Model attempt: Logistic regressor

In [12]:
warnings.filterwarnings("ignore")
training_data = preprocess_data(training_df)
X = training_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y = training_data['depression']

hyperparams = [
    {'penalty': 'l1', 'C': 1.0, 'solver': 'liblinear'},
    {'penalty': 'l2', 'C': 1.0, 'solver': 'liblinear'},
    {'penalty': 'l2', 'C': 0.99, 'solver': 'liblinear'},
    {'penalty': 'l2', 'C': 0.95, 'solver': 'liblinear'},
    {'penalty': 'l2', 'C': 0.9, 'solver': 'liblinear'},
    {'penalty': 'l2', 'C': 1.0, 'solver': 'lbfgs'},
    {'penalty': 'l2', 'C': 1.0, 'solver': 'sag'},
    {'penalty': 'l2', 'C': 1.0, 'solver': 'saga'},
    {'penalty': 'l2', 'C': 1.0, 'solver': 'newton-cg'},
    {'penalty': 'l2', 'C': 1.0, 'solver': 'newton-cholesky'},
]

for hyperparam in hyperparams:
    print(hyperparam)
    # Initialize Logistic regression model
    model = LogisticRegression(solver=hyperparam['solver'], C=hyperparam['C'], penalty=hyperparam['penalty'])
    # Cross validation k fold, 4:1::training:validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    #list to store metrics for each cross validation split
    fold_metrics = []
    
    # Perform cross-validation and collect metrics
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
        #Fit the training set
        model.fit(X_train, y_train)
        #Predict validation set
        y_pred = model.predict(X_val)
        #calculate metrics
        fold_metrics.append(calculate_metrics(y_val, y_pred, training_data.iloc[val_index]))
    
    #find avg metrics for each depth
    # metrics[depth] = {}
    sums = {}
    for metric in fold_metrics:
        for key, value in metric.items():
            sums[key] = sums.get(key, 0) + value
    for key in sums:
        sums[key]/=len(fold_metrics)
    print(sums)

{'penalty': 'l1', 'C': 1.0, 'solver': 'liblinear'}
{'Total accuracy': 0.7428256880733944, 'Total Balanced accuracy': 0.5884060803856384, 'Total EO': 0.9477892614269802, 'Aggregated accuracy score': 0.7609195402298852, 'Aggregated balanced accuracy score': 0.5666666666666667, 'Aggregated EO score': 0.9333333333333332, 'Aggregated balanced accuracy score (male)': 0.5666666666666667, 'Aggregated balanced accuracy score (female)': 0.5666666666666667}
{'penalty': 'l2', 'C': 1.0, 'solver': 'liblinear'}
{'Total accuracy': 0.7423119266055046, 'Total Balanced accuracy': 0.5882667934329493, 'Total EO': 0.9466991528254788, 'Aggregated accuracy score': 0.7586206896551724, 'Aggregated balanced accuracy score': 0.5625, 'Aggregated EO score': 0.9166666666666666, 'Aggregated balanced accuracy score (male)': 0.5666666666666667, 'Aggregated balanced accuracy score (female)': 0.5583333333333333}
{'penalty': 'l2', 'C': 0.99, 'solver': 'liblinear'}
{'Total accuracy': 0.7422385321100918, 'Total Balanced acc

In [13]:
# ---------------------------------------------------------Testing for best hyperparameter----------------------------------------------------------------
test_data = preprocess_data(test_df)
training_data = preprocess_data(training_df)
X_train = training_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y_train = training_data['depression']
X_test = test_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y_test = test_data['depression']

model = LogisticRegression(penalty='l1', C=1.0, solver='liblinear')
#Fit the training set
model.fit(X_train, y_train)
#Predict for test set
y_pred = model.predict(X_test)
#get metrics
metrics = calculate_metrics(y_test, y_pred, test_data)

print(metrics)

{'Total accuracy': 0.761890243902439, 'Total Balanced accuracy': 0.5125821245273796, 'Total EO': 0.9488448844884488, 'Aggregated accuracy score': 0.7, 'Aggregated balanced accuracy score': 0.5, 'Aggregated EO score': 1.0, 'Aggregated balanced accuracy score (male)': 0.5, 'Aggregated balanced accuracy score (female)': 0.5}


### Model attempt: Random Forest Classifier

In [14]:
training_data = preprocess_data(training_df)
X = training_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y = training_data['depression']

# Define a list of different numbers of trees to experiment with
num_trees_list = [5, 10, 15, 20, 50, 70, 100, 500, 1000]

# Initialize list to store results
metrics_rf = {}

# Iterate over each number of trees
for num_trees in num_trees_list:
    # Initialize model
    model = RandomForestClassifier(n_estimators=num_trees, random_state=42)
    # Cross validation k fold, 4:1::training:validation
    kf = KFold(n_splits=5, shuffle=True, random_state=1)
    #list to store metrics for each cross validation split
    fold_metrics = []

    # Perform cross-validation and collect metrics
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        #Fit the training set
        model.fit(X_train, y_train)
        #Predict validation set
        y_pred = model.predict(X_val)
        #calculate metrics
        fold_metrics.append(calculate_metrics(y_val, y_pred, training_data.iloc[val_index]))

    #find avg metrics for each depth
    metrics_rf[num_trees] = {}
    sums = {}
    for metric in fold_metrics:
        for key, value in metric.items():
            sums[key] = sums.get(key, 0) + value
    for key in sums:
        sums[key]/=len(fold_metrics)
    metrics_rf[num_trees] = sums

#Print metrics for all the hyperparameters (Depth)
for num_trees in metrics_rf:
    print(f"for num_trees {num_trees}")
    print(metrics_rf[num_trees])
    print()

for num_trees 5
{'Total accuracy': 0.7819449541284403, 'Total Balanced accuracy': 0.7000810674329243, 'Total EO': 0.9385183644605913, 'Aggregated accuracy score': 0.8482758620689654, 'Aggregated balanced accuracy score': 0.7301587301587301, 'Aggregated EO score': 0.8333333333333334, 'Aggregated balanced accuracy score (male)': 0.6916666666666667, 'Aggregated balanced accuracy score (female)': 0.7666666666666666}

for num_trees 10
{'Total accuracy': 0.8041100917431192, 'Total Balanced accuracy': 0.6930814358470765, 'Total EO': 0.931210586625507, 'Aggregated accuracy score': 0.8160919540229885, 'Aggregated balanced accuracy score': 0.6666666666666667, 'Aggregated EO score': 0.8000000000000002, 'Aggregated balanced accuracy score (male)': 0.625, 'Aggregated balanced accuracy score (female)': 0.7083333333333333}

for num_trees 15
{'Total accuracy': 0.8192293577981651, 'Total Balanced accuracy': 0.7253014640170138, 'Total EO': 0.938771098013295, 'Aggregated accuracy score': 0.84597701149425

In [15]:
# ---------------------------------------------------------Testing for best num_trees----------------------------------------------------------------
test_data = preprocess_data(test_df)
training_data = preprocess_data(training_df)
X_train = training_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y_train = training_data['depression']
X_test = test_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y_test = test_data['depression']

metrics = {}

best_num_trees = 500
model = RandomForestClassifier(n_estimators=best_num_trees, random_state=1)
#Fit the training set
model.fit(X_train, y_train)
#Predict for test set
y_pred = model.predict(X_test)
#get metrics
metrics = calculate_metrics(y_test, y_pred, test_data)

print(metrics)

{'Total accuracy': 0.7460365853658537, 'Total Balanced accuracy': 0.4824551819318852, 'Total EO': 0.7718734639421388, 'Aggregated accuracy score': 0.65, 'Aggregated balanced accuracy score': 0.4642857142857143, 'Aggregated EO score': 1.0, 'Aggregated balanced accuracy score (male)': 0.45454545454545453, 'Aggregated balanced accuracy score (female)': 0.5}


### Model attempt: AdaBoost Classifier

In [16]:
training_data = preprocess_data(training_df)
X = training_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y = training_data['depression']

# Define a list of different hyperparams of trees to experiment with
hyperparams = [
    {'n_estimators': 5, 'learning_rate': 1.0},
    {'n_estimators': 10, 'learning_rate': 1.0},
    {'n_estimators': 20, 'learning_rate': 1.0},
    {'n_estimators': 50, 'learning_rate': 1.0},
    {'n_estimators': 100, 'learning_rate': 1.0},
    {'n_estimators': 200, 'learning_rate': 1.0},
    {'n_estimators': 500, 'learning_rate': 1.0},
    {'n_estimators': 1000, 'learning_rate': 1.0},
    {'n_estimators': 100, 'learning_rate': 0.1},
    {'n_estimators': 100, 'learning_rate': 0.5},
]


# Iterate over each number of trees
for hyperparam in hyperparams:
    print(hyperparam)
    # Initialize model
    model = AdaBoostClassifier(estimator=None, n_estimators=hyperparam['n_estimators'], learning_rate=hyperparam['learning_rate'], random_state=42)
    # Cross validation k fold, 4:1::training:validation
    kf = KFold(n_splits=5, shuffle=True, random_state=1)
    #list to store metrics for each cross validation split
    fold_metrics = []

    # Perform cross-validation and collect metrics
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        #Fit the training set
        model.fit(X_train, y_train)
        #Predict validation set
        y_pred = model.predict(X_val)
        #calculate metrics
        fold_metrics.append(calculate_metrics(y_val, y_pred, training_data.iloc[val_index]))

    #find avg metrics for each depth
    sums = {}
    for metric in fold_metrics:
        for key, value in metric.items():
            sums[key] = sums.get(key, 0) + value
    for key in sums:
        sums[key]/=len(fold_metrics)
    print(sums)

{'n_estimators': 5, 'learning_rate': 1.0}
{'Total accuracy': 0.7305688073394496, 'Total Balanced accuracy': 0.5573661655081127, 'Total EO': 0.9480545632430669, 'Aggregated accuracy score': 0.7471264367816092, 'Aggregated balanced accuracy score': 0.5416666666666666, 'Aggregated EO score': 0.8333333333333334, 'Aggregated balanced accuracy score (male)': 0.5833333333333334, 'Aggregated balanced accuracy score (female)': 0.5}
{'n_estimators': 10, 'learning_rate': 1.0}
{'Total accuracy': 0.729467889908257, 'Total Balanced accuracy': 0.5635483158212631, 'Total EO': 0.9452287809099952, 'Aggregated accuracy score': 0.7448275862068966, 'Aggregated balanced accuracy score': 0.5374999999999999, 'Aggregated EO score': 0.85, 'Aggregated balanced accuracy score (male)': 0.5750000000000001, 'Aggregated balanced accuracy score (female)': 0.5}
{'n_estimators': 20, 'learning_rate': 1.0}
{'Total accuracy': 0.7310825688073395, 'Total Balanced accuracy': 0.5759445365113705, 'Total EO': 0.9480179997505545,

In [17]:
# ---------------------------------------------------------Testing for best hyperparameter----------------------------------------------------------------
test_data = preprocess_data(test_df)
training_data = preprocess_data(training_df)
X_train = training_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y_train = training_data['depression']
X_test = test_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y_test = test_data['depression']

# Initialize decision tree model
model = AdaBoostClassifier(estimator=None, n_estimators=1000, learning_rate=1.0, random_state=42)
#Fit the training set
model.fit(X_train, y_train)
#Predict for test set
y_pred = model.predict(X_test)
#get metrics
metrics = calculate_metrics(y_test, y_pred, test_data)

print(metrics)

{'Total accuracy': 0.6344512195121951, 'Total Balanced accuracy': 0.5610532066303426, 'Total EO': 0.7849343444982797, 'Aggregated accuracy score': 0.6, 'Aggregated balanced accuracy score': 0.47619047619047616, 'Aggregated EO score': 0.8, 'Aggregated balanced accuracy score (male)': 0.36363636363636365, 'Aggregated balanced accuracy score (female)': 0.6}


### Model attempt: Linear Perceptron

In [18]:
warnings.filterwarnings("ignore")
training_data = preprocess_data(training_df)
X = training_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y = training_data['depression']

hyperparams = [
    {'penalty': 'l1'},
    {'penalty': 'l2'},
    {'penalty': None}
]

for hyperparam in hyperparams:
    print(hyperparam)
    # Initialize Logistic regression model
    model = Perceptron(penalty=hyperparam['penalty'])
    # Cross validation k fold, 4:1::training:validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    #list to store metrics for each cross validation split
    fold_metrics = []
    
    # Perform cross-validation and collect metrics
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
        #Fit the training set
        model.fit(X_train, y_train)
        #Predict validation set
        y_pred = model.predict(X_val)
        #calculate metrics
        fold_metrics.append(calculate_metrics(y_val, y_pred, training_data.iloc[val_index]))
    
    #find avg metrics for each depth
    # metrics[depth] = {}
    sums = {}
    for metric in fold_metrics:
        for key, value in metric.items():
            sums[key] = sums.get(key, 0) + value
    for key in sums:
        sums[key]/=len(fold_metrics)
    print(sums)

{'penalty': 'l1'}
{'Total accuracy': 0.6464587155963304, 'Total Balanced accuracy': 0.5681661948483416, 'Total EO': 0.9466788052818206, 'Aggregated accuracy score': 0.7149425287356321, 'Aggregated balanced accuracy score': 0.5890873015873016, 'Aggregated EO score': 0.7833333333333333, 'Aggregated balanced accuracy score (male)': 0.5538461538461539, 'Aggregated balanced accuracy score (female)': 0.6333333333333334}
{'penalty': 'l2'}
{'Total accuracy': 0.6399266055045871, 'Total Balanced accuracy': 0.5727839452622211, 'Total EO': 0.9527539484807953, 'Aggregated accuracy score': 0.7057471264367816, 'Aggregated balanced accuracy score': 0.577579365079365, 'Aggregated EO score': 0.6166666666666667, 'Aggregated balanced accuracy score (male)': 0.5326923076923077, 'Aggregated balanced accuracy score (female)': 0.5958333333333333}
{'penalty': None}
{'Total accuracy': 0.6422752293577981, 'Total Balanced accuracy': 0.5640207194817426, 'Total EO': 0.9474830249543057, 'Aggregated accuracy score': 

In [19]:
# ---------------------------------------------------------Testing for best hyperparameter----------------------------------------------------------------
test_data = preprocess_data(test_df)
training_data = preprocess_data(training_df)
X_train = training_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y_train = training_data['depression']
X_test = test_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y_test = test_data['depression']

model = Perceptron(penalty='l1')
#Fit the training set
model.fit(X_train, y_train)
#Predict for test set
y_pred = model.predict(X_test)
#get metrics
metrics = calculate_metrics(y_test, y_pred, test_data)

print(metrics)

{'Total accuracy': 0.49817073170731707, 'Total Balanced accuracy': 0.44699032476027545, 'Total EO': 0.8446738290850362, 'Aggregated accuracy score': 0.4, 'Aggregated balanced accuracy score': 0.38095238095238093, 'Aggregated EO score': 0.19999999999999996, 'Aggregated balanced accuracy score (male)': 0.6818181818181819, 'Aggregated balanced accuracy score (female)': 0.43333333333333335}


## Gender Classification

### Model attempt: Decision Tree

In [20]:
#------------------------------------------------Tuning for different depths-------------------------------------------------------

warnings.filterwarnings("ignore")
training_data = preprocess_data(training_df)
X = training_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y = training_data['gender']

# Define the depths to experiment with
depths = [3, 5, 7, 9, 15, 30, 50, 70, 90]
# Initialize metrics
metrics = {}

# Perform cross-validation for each tree depth
for depth in depths:
    # Initialize decision tree model
    tree = DecisionTreeClassifier(max_depth=depth, criterion='entropy')
    # Cross validation k fold, 4:1::training:validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    #list to store metrics for each cross validation split
    fold_metrics = []

    # Perform cross-validation and collect metrics
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        #Fit the training set
        tree.fit(X_train, y_train)
        #Predict validation set
        y_pred = tree.predict(X_val)
        #calculate metrics
        fold_metrics.append(calculate_metrics(y_val, y_pred, training_data.iloc[val_index], False))

    #find avg metrics for each depth
    metrics[depth] = {}
    sums = {}
    for metric in fold_metrics:
        for key, value in metric.items():
            sums[key] = sums.get(key, 0) + value
    for key in sums:
        sums[key]/=len(fold_metrics)
    metrics[depth] = sums

#Print metrics for all the hyperparameters (Depth)
for depth in metrics:
    print(f"for depth {depth}")
    print(metrics[depth])
    print()

for depth 3
{'Total accuracy': 0.9236697247706422, 'Total Balanced accuracy': 0.9192004514666632, 'Aggregated accuracy score': 0.9862068965517242, 'Aggregated balanced accuracy score': 0.9841503267973856}

for depth 5
{'Total accuracy': 0.9288073394495413, 'Total Balanced accuracy': 0.9226857638489625, 'Aggregated accuracy score': 0.993103448275862, 'Aggregated balanced accuracy score': 0.9916666666666668}

for depth 7
{'Total accuracy': 0.9288073394495413, 'Total Balanced accuracy': 0.9257402712308427, 'Aggregated accuracy score': 0.9954022988505746, 'Aggregated balanced accuracy score': 0.9944444444444445}

for depth 9
{'Total accuracy': 0.927045871559633, 'Total Balanced accuracy': 0.9238335826876496, 'Aggregated accuracy score': 0.9954022988505746, 'Aggregated balanced accuracy score': 0.9944444444444445}

for depth 15
{'Total accuracy': 0.924697247706422, 'Total Balanced accuracy': 0.9218074709566804, 'Aggregated accuracy score': 0.993103448275862, 'Aggregated balanced accuracy sc

In [21]:
# ---------------------------------------------------------Testing for best depth----------------------------------------------------------------
test_data = preprocess_data(test_df)
training_data = preprocess_data(training_df)
X_train = training_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y_train = training_data['gender']
X_test = test_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y_test = test_data['gender']

best_depth = 70
# Initialize decision tree model
tree = DecisionTreeClassifier(max_depth=best_depth, criterion='entropy')
#Fit the training set
tree.fit(X_train, y_train)
#Predict for test set
y_pred = tree.predict(X_test)
#get metrics
metrics = calculate_metrics(y_test, y_pred, test_data, False)

print(metrics)

{'Total accuracy': 0.8634146341463415, 'Total Balanced accuracy': 0.8574221257082024, 'Aggregated accuracy score': 0.95, 'Aggregated balanced accuracy score': 0.9375}


### Model attempt: Logistic regressor

In [22]:
warnings.filterwarnings("ignore")
training_data = preprocess_data(training_df)
X = training_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y = training_data['gender']

hyperparams = [
    {'penalty': 'l1', 'C': 1.0, 'solver': 'liblinear'},
    {'penalty': 'l2', 'C': 1.0, 'solver': 'liblinear'},
    {'penalty': 'l2', 'C': 0.99, 'solver': 'liblinear'},
    {'penalty': 'l2', 'C': 0.95, 'solver': 'liblinear'},
    {'penalty': 'l2', 'C': 0.9, 'solver': 'liblinear'},
    {'penalty': 'l2', 'C': 1.0, 'solver': 'lbfgs'},
    {'penalty': 'l2', 'C': 1.0, 'solver': 'sag'},
    {'penalty': 'l2', 'C': 1.0, 'solver': 'saga'},
    {'penalty': 'l2', 'C': 1.0, 'solver': 'newton-cg'},
    {'penalty': 'l2', 'C': 1.0, 'solver': 'newton-cholesky'},
]

for hyperparam in hyperparams:
    print(hyperparam)
    # Initialize Logistic regression model
    model = LogisticRegression(solver=hyperparam['solver'], C=hyperparam['C'], penalty=hyperparam['penalty'])
    # Cross validation k fold, 4:1::training:validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    #list to store metrics for each cross validation split
    fold_metrics = []
    
    # Perform cross-validation and collect metrics
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
        #Fit the training set
        model.fit(X_train, y_train)
        #Predict validation set
        y_pred = model.predict(X_val)
        #calculate metrics
        fold_metrics.append(calculate_metrics(y_val, y_pred, training_data.iloc[val_index], False))
    
    #find avg metrics for each depth
    # metrics[depth] = {}
    sums = {}
    for metric in fold_metrics:
        for key, value in metric.items():
            sums[key] = sums.get(key, 0) + value
    for key in sums:
        sums[key]/=len(fold_metrics)
    print(sums)

{'penalty': 'l1', 'C': 1.0, 'solver': 'liblinear'}
{'Total accuracy': 0.9479633027522937, 'Total Balanced accuracy': 0.9445513343092851, 'Aggregated accuracy score': 1.0, 'Aggregated balanced accuracy score': 1.0}
{'penalty': 'l2', 'C': 1.0, 'solver': 'liblinear'}
{'Total accuracy': 0.9472293577981651, 'Total Balanced accuracy': 0.9434794099222874, 'Aggregated accuracy score': 0.9977011494252874, 'Aggregated balanced accuracy score': 0.9972222222222221}
{'penalty': 'l2', 'C': 0.99, 'solver': 'liblinear'}
{'Total accuracy': 0.9472293577981651, 'Total Balanced accuracy': 0.9434794099222874, 'Aggregated accuracy score': 0.9977011494252874, 'Aggregated balanced accuracy score': 0.9972222222222221}
{'penalty': 'l2', 'C': 0.95, 'solver': 'liblinear'}
{'Total accuracy': 0.9471559633027521, 'Total Balanced accuracy': 0.9434183225673698, 'Aggregated accuracy score': 0.9977011494252874, 'Aggregated balanced accuracy score': 0.9972222222222221}
{'penalty': 'l2', 'C': 0.9, 'solver': 'liblinear'}
{

In [23]:
# ---------------------------------------------------------Testing for best hyperparameter----------------------------------------------------------------
test_data = preprocess_data(test_df)
training_data = preprocess_data(training_df)
X_train = training_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y_train = training_data['gender']
X_test = test_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y_test = test_data['gender']

model = LogisticRegression(penalty='l1', C=1.0, solver='liblinear')
#Fit the training set
model.fit(X_train, y_train)
#Predict for test set
y_pred = model.predict(X_test)
#get metrics
metrics = calculate_metrics(y_test, y_pred, test_data, False)

print(metrics)

{'Total accuracy': 0.8759146341463414, 'Total Balanced accuracy': 0.8733573414574689, 'Aggregated accuracy score': 0.9, 'Aggregated balanced accuracy score': 0.8958333333333333}


### Model attempt: Random forest classifier

In [24]:
training_data = preprocess_data(training_df)
X = training_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y = training_data['gender']

# Define a list of different numbers of trees to experiment with
num_trees_list = [5, 10, 15, 20, 50, 70, 100, 500, 1000]

# Initialize list to store results
metrics_rf = {}

# Iterate over each number of trees
for num_trees in num_trees_list:
    # Initialize model
    model = RandomForestClassifier(n_estimators=num_trees, random_state=42)
    # Cross validation k fold, 4:1::training:validation
    kf = KFold(n_splits=5, shuffle=True, random_state=1)
    #list to store metrics for each cross validation split
    fold_metrics = []

    # Perform cross-validation and collect metrics
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        #Fit the training set
        model.fit(X_train, y_train)
        #Predict validation set
        y_pred = model.predict(X_val)
        #calculate metrics
        fold_metrics.append(calculate_metrics(y_val, y_pred, training_data.iloc[val_index], False))

    #find avg metrics for each depth
    metrics_rf[num_trees] = {}
    sums = {}
    for metric in fold_metrics:
        for key, value in metric.items():
            sums[key] = sums.get(key, 0) + value
    for key in sums:
        sums[key]/=len(fold_metrics)
    metrics_rf[num_trees] = sums

#Print metrics for all the hyperparameters (Depth)
for num_trees in metrics_rf:
    print(f"for num_trees {num_trees}")
    print(metrics_rf[num_trees])
    print()

for num_trees 5
{'Total accuracy': 0.9354128440366974, 'Total Balanced accuracy': 0.932551379725797, 'Aggregated accuracy score': 0.993103448275862, 'Aggregated balanced accuracy score': 0.9916666666666668}

for num_trees 10
{'Total accuracy': 0.9395229357798165, 'Total Balanced accuracy': 0.9390879733703412, 'Aggregated accuracy score': 0.993103448275862, 'Aggregated balanced accuracy score': 0.9916666666666668}

for num_trees 15
{'Total accuracy': 0.9431192660550458, 'Total Balanced accuracy': 0.9402023270255062, 'Aggregated accuracy score': 0.9908045977011494, 'Aggregated balanced accuracy score': 0.9888888888888889}

for num_trees 20
{'Total accuracy': 0.944, 'Total Balanced accuracy': 0.9423465197030889, 'Aggregated accuracy score': 0.993103448275862, 'Aggregated balanced accuracy score': 0.9916666666666668}

for num_trees 50
{'Total accuracy': 0.9478899082568807, 'Total Balanced accuracy': 0.9463822431074927, 'Aggregated accuracy score': 0.993103448275862, 'Aggregated balanced ac

In [25]:
# ---------------------------------------------------------Testing for best num_trees----------------------------------------------------------------
test_data = preprocess_data(test_df)
training_data = preprocess_data(training_df)
X_train = training_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y_train = training_data['gender']
X_test = test_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y_test = test_data['gender']

metrics = {}

best_num_trees = 500
# Initialize decision tree model
model = RandomForestClassifier(n_estimators=best_num_trees, random_state=42)
#Fit the training set
model.fit(X_train, y_train)
#Predict for test set
y_pred = model.predict(X_test)
#get metrics
metrics = calculate_metrics(y_test, y_pred, test_data, False)

print(metrics)

{'Total accuracy': 0.8975609756097561, 'Total Balanced accuracy': 0.8925856270787773, 'Aggregated accuracy score': 0.95, 'Aggregated balanced accuracy score': 0.9375}


### Model attempt: Adaboost classifier

In [26]:
training_data = preprocess_data(training_df)
X = training_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y = training_data['gender']

# Define a list of different hyperparams of trees to experiment with
hyperparams = [
    {'n_estimators': 5, 'learning_rate': 1.0},
    {'n_estimators': 10, 'learning_rate': 1.0},
    {'n_estimators': 20, 'learning_rate': 1.0},
    {'n_estimators': 50, 'learning_rate': 1.0},
    {'n_estimators': 100, 'learning_rate': 1.0},
    {'n_estimators': 200, 'learning_rate': 1.0},
    {'n_estimators': 500, 'learning_rate': 1.0},
    {'n_estimators': 1000, 'learning_rate': 1.0},
    {'n_estimators': 100, 'learning_rate': 0.1},
    {'n_estimators': 100, 'learning_rate': 0.5},
]


# Iterate over each number of trees
for hyperparam in hyperparams:
    print(hyperparam)
    # Initialize model
    model = AdaBoostClassifier(estimator=None, n_estimators=hyperparam['n_estimators'], learning_rate=hyperparam['learning_rate'], random_state=42)
    # Cross validation k fold, 4:1::training:validation
    kf = KFold(n_splits=5, shuffle=True, random_state=1)
    #list to store metrics for each cross validation split
    fold_metrics = []

    # Perform cross-validation and collect metrics
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        #Fit the training set
        model.fit(X_train, y_train)
        #Predict validation set
        y_pred = model.predict(X_val)
        #calculate metrics
        fold_metrics.append(calculate_metrics(y_val, y_pred, training_data.iloc[val_index], False))

    #find avg metrics for each depth
    sums = {}
    for metric in fold_metrics:
        for key, value in metric.items():
            sums[key] = sums.get(key, 0) + value
    for key in sums:
        sums[key]/=len(fold_metrics)
    print(sums)

{'n_estimators': 5, 'learning_rate': 1.0}
{'Total accuracy': 0.9187522935779817, 'Total Balanced accuracy': 0.9132265743408574, 'Aggregated accuracy score': 0.9793103448275863, 'Aggregated balanced accuracy score': 0.975}
{'n_estimators': 10, 'learning_rate': 1.0}
{'Total accuracy': 0.9202935779816513, 'Total Balanced accuracy': 0.9134970912682283, 'Aggregated accuracy score': 0.9770114942528736, 'Aggregated balanced accuracy score': 0.9722222222222221}
{'n_estimators': 20, 'learning_rate': 1.0}
{'Total accuracy': 0.9342385321100919, 'Total Balanced accuracy': 0.9289966349024745, 'Aggregated accuracy score': 0.993103448275862, 'Aggregated balanced accuracy score': 0.9916666666666668}
{'n_estimators': 50, 'learning_rate': 1.0}
{'Total accuracy': 0.943045871559633, 'Total Balanced accuracy': 0.9387751418931648, 'Aggregated accuracy score': 0.9954022988505746, 'Aggregated balanced accuracy score': 0.9944444444444445}
{'n_estimators': 100, 'learning_rate': 1.0}
{'Total accuracy': 0.9460550

In [27]:
# ---------------------------------------------------------Testing for best hyperparameter----------------------------------------------------------------
test_data = preprocess_data(test_df)
training_data = preprocess_data(training_df)
X_train = training_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y_train = training_data['gender']
X_test = test_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y_test = test_data['gender']

# Initialize decision tree model
model = AdaBoostClassifier(estimator=None, n_estimators=500, learning_rate=1.0, random_state=42)
#Fit the training set
model.fit(X_train, y_train)
#Predict for test set
y_pred = model.predict(X_test)
#get metrics
metrics = calculate_metrics(y_test, y_pred, test_data, False)

print(metrics)

{'Total accuracy': 0.8829268292682927, 'Total Balanced accuracy': 0.881426716928279, 'Aggregated accuracy score': 0.9, 'Aggregated balanced accuracy score': 0.8958333333333333}


### Model attempt: Linear Perceptron

In [28]:
warnings.filterwarnings("ignore")
training_data = preprocess_data(training_df)
X = training_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y = training_data['gender']

hyperparams = [
    {'penalty': 'l1'},
    {'penalty': 'l2'},
    {'penalty': None}
]

for hyperparam in hyperparams:
    print(hyperparam)
    # Initialize Logistic regression model
    model = Perceptron(penalty=hyperparam['penalty'])
    # Cross validation k fold, 4:1::training:validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    #list to store metrics for each cross validation split
    fold_metrics = []
    
    # Perform cross-validation and collect metrics
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
        #Fit the training set
        model.fit(X_train, y_train)
        #Predict validation set
        y_pred = model.predict(X_val)
        #calculate metrics
        fold_metrics.append(calculate_metrics(y_val, y_pred, training_data.iloc[val_index], False))
    
    #find avg metrics for each depth
    # metrics[depth] = {}
    sums = {}
    for metric in fold_metrics:
        for key, value in metric.items():
            sums[key] = sums.get(key, 0) + value
    for key in sums:
        sums[key]/=len(fold_metrics)
    print(sums)

{'penalty': 'l1'}
{'Total accuracy': 0.9233027522935779, 'Total Balanced accuracy': 0.920602012965162, 'Aggregated accuracy score': 0.9816091954022989, 'Aggregated balanced accuracy score': 0.981045751633987}
{'penalty': 'l2'}
{'Total accuracy': 0.9127339449541283, 'Total Balanced accuracy': 0.9096748165836269, 'Aggregated accuracy score': 0.9816091954022989, 'Aggregated balanced accuracy score': 0.9794117647058822}
{'penalty': None}
{'Total accuracy': 0.9219816513761467, 'Total Balanced accuracy': 0.9184822491797735, 'Aggregated accuracy score': 0.9908045977011495, 'Aggregated balanced accuracy score': 0.9905228758169935}


In [29]:
# ---------------------------------------------------------Testing for best hyperparameter----------------------------------------------------------------
test_data = preprocess_data(test_df)
training_data = preprocess_data(training_df)
X_train = training_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y_train = training_data['gender']
X_test = test_data.drop(columns = ['participant', 'gender', 'depression'], axis=1)
y_test = test_data['gender']

# Initialize decision tree model
model = Perceptron(penalty=None)
#Fit the training set
model.fit(X_train, y_train)
#Predict for test set
y_pred = model.predict(X_test)
#get metrics
metrics = calculate_metrics(y_test, y_pred, test_data, False)

print(metrics)

{'Total accuracy': 0.8774390243902439, 'Total Balanced accuracy': 0.8808123821625446, 'Aggregated accuracy score': 1.0, 'Aggregated balanced accuracy score': 1.0}


## Current model performance
### Depression classification
- Linear Perceptron (Penalty: l1)
  {'Total accuracy': 0.49817073170731707, 'Total Balanced accuracy': 0.44699032476027545, 'Total EO': 0.8446738290850362, 'Aggregated accuracy score': 0.4, 'Aggregated balanced accuracy score': 0.38095238095238093, 'Aggregated EO score': 0.19999999999999996, 'Aggregated balanced accuracy score (male)': 0.6818181818181819, 'Aggregated balanced accuracy score (female)': 0.43333333333333335}
- Logistic Regression (Penalty: l1, C:1.0, solver: liblinear)
  {'Total accuracy': 0.761890243902439, 'Total Balanced accuracy': 0.5125821245273796, 'Total EO': 0.9488448844884488, 'Aggregated accuracy score': 0.7, 'Aggregated balanced accuracy score': 0.5, 'Aggregated EO score': 1.0, 'Aggregated balanced accuracy score (male)': 0.5, 'Aggregated balanced accuracy score (female)': 0.5}
- Decision Tree Classifier (Depth:70)
  **{'Total accuracy': 0.688109756097561, 'Total Balanced accuracy': 0.5868557627337944, 'Total EO': 0.7612878309107507, 'Aggregated accuracy score': 0.7, 'Aggregated balanced accuracy score': 0.5476190476190477, 'Aggregated EO score': 0.8, 'Aggregated balanced accuracy score (male)': 0.45454545454545453, 'Aggregated balanced accuracy score (female)': 0.6}**
- Random forest classifier (num trees: 500)
  {'Total accuracy': 0.7460365853658537, 'Total Balanced accuracy': 0.4824551819318852, 'Total EO': 0.7718734639421388, 'Aggregated accuracy score': 0.65, 'Aggregated balanced accuracy score': 0.4642857142857143, 'Aggregated EO score': 1.0, 'Aggregated balanced accuracy score (male)': 0.45454545454545453, 'Aggregated balanced accuracy score (female)': 0.5}
- Adaboost classifier (estimators: 1000, learning rate: 1.0)
{'Total accuracy': 0.6344512195121951, 'Total Balanced accuracy': 0.5610532066303426, 'Total EO': 0.7849343444982797, 'Aggregated accuracy score': 0.6, 'Aggregated balanced accuracy score': 0.47619047619047616, 'Aggregated EO score': 0.8, 'Aggregated balanced accuracy score (male)': 0.36363636363636365, 'Aggregated balanced accuracy score (female)': 0.6}


### Gender classification
- Linear Perceptron (Penalty: None)
  **{'Total accuracy': 0.8774390243902439, 'Total Balanced accuracy': 0.8808123821625446, 'Aggregated accuracy score': 1.0, 'Aggregated balanced accuracy score': 1.0}**
- Logistic Regression (Penalty: l1, C:1.0, solver: liblinear)
  {'Total accuracy': 0.876219512195122, 'Total Balanced accuracy': 0.8736052343677317, 'Aggregated accuracy score': 0.9, 'Aggregated balanced accuracy score': 0.8958333333333333}
- Decision Tree Classifier (Depth:70)
  {'Total accuracy': 0.864329268292683, 'Total Balanced accuracy': 0.856241935629493, 'Aggregated accuracy score': 0.95, 'Aggregated balanced accuracy score': 0.9375}
- Random forest classifier (num trees: 500)
  {'Total accuracy': 0.8975609756097561, 'Total Balanced accuracy': 0.8925856270787773, 'Aggregated accuracy score': 0.95, 'Aggregated balanced accuracy score': 0.9375}
- Adaboost Classifier (estimators: 500, learning rate: 1.0)
{'Total accuracy': 0.8829268292682927, 'Total Balanced accuracy': 0.881426716928279, 'Aggregated accuracy score': 0.9, 'Aggregated balanced accuracy score': 0.8958333333333333}